In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@{BRANCH}#egg=nemo_toolkit[nlp]

In [4]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
[NeMo W 2020-09-10 14:56:52 experimental:28] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-09-10 14:56:52 experimental:28] Module <class 'nemo.collections.nlp.modules.common.sequence_token_classifier.SequenceTokenClassifier'> is experimental, not ready for production and is not fully supported. Use at your own risk.


# Task Description
**Joint Intent and Slot classification** - is a task of classifying an Intent and detecting all relevant Slots (Entities)
for this Intent in a query.
For example, in the query:  `What is the weather in Santa Clara tomorrow morning?`, we would like to classify the query
as `Weather` Intent, and detect `Santa Clara` as a `location` slot and `tomorrow morning` as a `date_time` slot.
Intents and Slots names are usually task specific and defined as labels in the training data.
This is a fundamental step that is executed in any task-driven Conversational Assistant.

Our Bert based model implementation enables to train and then detect both of these tasks together.


# Dataset and Nemo data format

In this tutorial we are going to use a virtual assistant interaction data set that can be downloaded from here: https://github.com/xliuhw/NLU-Evaluation-Data.
There are about 10K training and 1K testing queries which cover 64 various Intents and 55 Slots. 

To work with Nemo NLP classification model, this dataset should be first converted to the Nemo format, which requires next files:
- **dict.intents.csv** - list of all intent names in the data. One line per an intent name.
- **dict.slots.csv** - list of all slot names in the data. One line per a slot name. It is possible to use both: B- I- notations, for separating between first and intermediate tokens for multi token slots. Or just use one slot type for each token of multi token slot. Our recommendation is to use later one, since it is simpler and there is no visible degradation in performance.
- **train.tsv/test.tsv** - contain original queries, one per line, and intent number separated by tab. For example: `what alarms do i have set right now	0`. Intent numbers are according to the intent line in the intent dictionary file (dict.intents.csv) starting from 0. First line of these files contains a header line: `sentence \tab label`.
- **train_slot.tvs/test_slot.tsv** - contain one line per a query, where instead each token there is a number of the token from the slots dictionary file (dict.slots.csv), starting from 0. Last 'out-of scope' token is usually located in the last line of the dictionary. Example: `54 0 0 54 54 12 12` (numbers separated by space). No header line in these files.

Nemo provides **import_dataset.py** converter for few reference datasets (Assistant/Atis/Snips) which converts them to Nemo data format for Intent and Slot classification model. If you have your own annotated dataset in different format, you will need to write a data converter. Possible recommended format for your own annotation, is to have one text file per all examples of one intent. With one line per query in a form like: `did i set an alarm to [alarm_type : wake up] in the [timeofday : morning]`, using brackets to define slot names. This very similar to Assistant format and you can use it's converter to Nemo format with small changes. 

You can run this utility as follows:

**python examples/nlp/intent_slot_classification/data/import_datasets.py --dataset_name=assistant --source_data_dir=source_dir_name --target_data_dir=target_dir_name**


# Download, preprocess and explore the dataset

In [2]:
# replace DATA_DIR and NEMO_DIR with a location of your data and nemo folders
DATA_DIR = "DATA_DIR"
NEMO_DIR = "NEMO_DIR"
EXAMPLE_DIR = f'{NEMO_DIR}/examples/nlp/intent_slot_classification'

In [15]:
# download and unzip example dataset from github
print('Downloading dataset...')
wget.download('https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip', DATA_DIR)
! unzip {DATA_DIR}/NLU-Evaluation-Data-master.zip -d {DATA_DIR}

Archive:  /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master.zip
c2491de8342748f9484896e48f21b85e59b3c4a2
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/AnnotatedData/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/AnnotatedData/NLU-Data-Home-Domain-Annotated-All.csv  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/Collected-Original-Data/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/Collected-Original-Data/paraphrases_and_intents_26k_normalised_all.csv  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/auto

  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/annotated/recommendation_movies/recommendation_movies_anno_20180322130133773.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/annotated/social_post/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/annotated/social_post/social_post_anno_20180322130133838.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/annotated/social_query/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidat

  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/text/general_praise.txt  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/text/general_quirky.txt  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/text/general_repeat.txt  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/text/iot_cleaning.txt  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/testset/tex

  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/trainset/iot_hue_lightoff.csv  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/trainset/iot_hue_lighton.csv  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/trainset/iot_hue_lightup.csv  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/trainset/iot_wemo_off.csv  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_1/trainset/iot_wemo_on.cs

  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/testset/annotated/transport_taxi/transport_taxi_anno_20180322130135651.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/testset/annotated/transport_ticket/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/testset/annotated/transport_ticket/transport_ticket_anno_20180322130135664.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossValidation/KFold_10/testset/annotated/transport_traffic/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/Cr

...

  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/email_sendemail.json  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/general_affirm.json  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/general_commandstop.json  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_9/merged/intents/general_confirm.json  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4ApiaiReal/Apiai_trainset_2018_03_22-13_01_25_169/CrossValidation/KFo

  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_10/merged/Luis_KFold_10_2018_03_22-13_01_25_169_20180327170202385.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_2/
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_2/merged/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_2/merged/Luis_KFold_2_2018_03_22-13_01_25_169_20180327170202385.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4LuisReal/Luis_trainset_2018_03_22-13_01_25_169/CrossValidation/KFold_3/
   creatin

   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/mergedTestset/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/mergedTestset/RasaNluTestset_Merged_2018_03_22-13_01_25_169.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/rasaTrainConfig/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_2/rasaTrainConfig/configLatest_KFold_2_mitie_2018_03_22-13_01_25_169.json  
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169

  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_7/merged/RasaNlu_Merged_KFold_7_2018_03_22-13_01_25_169.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_7/mergedTestset/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_7/mergedTestset/RasaNluTestset_Merged_2018_03_22-13_01_25_169.json  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/CrossValidation/KFold_7/rasaTrainConfig/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4RasaReal/rasa_json_2018_03_22-13_01_25_169_80Train/Cros

   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_6/
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_6/merged/
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_6/merged/entities/
  inflating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_6/merged/entities/watsonEntities_2018_03_22-13_01_25_169.csv  
   creating: /Users/vgetselevich/Development/data/NLU-Evaluation-Data-master/CrossValidation/out4WatsonReal/Watson_2018_03_22-13_01_25_169_trainset/CrossValidation/KFold_6/merged/intents/
  inflating: /Users/vgetselevich/Deve

In [51]:
# convert the dataset to Nemo format
!python {EXAMPLE_DIR}/data/import_datasets.py --dataset_name=assistant --source_data_dir={DATA_DIR}/NLU-Evaluation-Data-master --target_data_dir={DATA_DIR}/nemo_format


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
[NeMo W 2020-09-10 14:21:49 experimental:28] Module <class 'nemo.collections.nlp.modules.common.megatron.megatron_bert.MegatronBertEncoder'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-09-10 14:21:49 experimental:28] Module <class 'nemo.collections.nlp.modules.common.sequence_token_classifier.SequenceTokenClassifier'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2020-09-10 14:21:49 assistant_utils:124] robot dataset has already been processed and stored at /Users/vgetselevich/Development/data/nemo_format


In [45]:
# list of queries divided by intent files in the original training dataset
! ls -l {DATA_DIR}/NLU-Evaluation-Data-master/dataset/trainset

total 2976
-rw-r--r--  1 vgetselevich  staff  22459 Apr 30  2019 alarm_query.csv
-rw-r--r--  1 vgetselevich  staff  13045 Apr 30  2019 alarm_remove.csv
-rw-r--r--  1 vgetselevich  staff  23552 Apr 30  2019 alarm_set.csv
-rw-r--r--  1 vgetselevich  staff   7911 Apr 30  2019 audio_volume_down.csv
-rw-r--r--  1 vgetselevich  staff  14525 Apr 30  2019 audio_volume_mute.csv
-rw-r--r--  1 vgetselevich  staff  14732 Apr 30  2019 audio_volume_up.csv
-rw-r--r--  1 vgetselevich  staff  27361 Apr 30  2019 calendar_query.csv
-rw-r--r--  1 vgetselevich  staff  27068 Apr 30  2019 calendar_remove.csv
-rw-r--r--  1 vgetselevich  staff  34492 Apr 30  2019 calendar_set.csv
-rw-r--r--  1 vgetselevich  staff  25138 Apr 30  2019 cooking_recipe.csv
-rw-r--r--  1 vgetselevich  staff  16357 Apr 30  2019 datetime_convert.csv
-rw-r--r--  1 vgetselevich  staff  23226 Apr 30  2019 datetime_query.csv
-rw-r--r--  1 vgetselevich  staff  12869 Apr 30  2019 email_addcontact.csv
-rw-r--r--  1 vgetselevich  staff  23510

In [41]:
# print all intents from nemo format intent dictionary
!echo 'Intents: ' $(wc -l < {DATA_DIR}/nemo_format/dict.intents.csv)
! cat {DATA_DIR}/nemo_format/dict.intents.csv

Intents:  64
alarm_query
alarm_remove
alarm_set
audio_volume_down
audio_volume_mute
audio_volume_up
calendar_query
calendar_remove
calendar_set
cooking_recipe
datetime_convert
datetime_query
email_addcontact
email_query
email_querycontact
email_sendemail
general_affirm
general_commandstop
general_confirm
general_dontcare
general_explain
general_joke
general_negate
general_praise
general_quirky
general_repeat
iot_cleaning
iot_coffee
iot_hue_lightchange
iot_hue_lightdim
iot_hue_lightoff
iot_hue_lighton
iot_hue_lightup
iot_wemo_off
iot_wemo_on
lists_createoradd
lists_query
lists_remove
music_likeness
music_query
music_settings
news_query
play_audiobook
play_game
play_music
play_podcasts
play_radio
qa_currency
qa_definition
qa_factoid
qa_maths
qa_stock
recommendation_events
recommendation_locations
recommendation_movies
social_post
social_query
takeaway_order
takeaway_query
transport_query
transport_taxi
transport_ticket
transport_traffic
weather_query


In [42]:
# print all slots from nemo format slot dictionary
!echo 'Slots: ' $(wc -l < {DATA_DIR}/nemo_format/dict.slots.csv)
! cat {DATA_DIR}/nemo_format/dict.slots.csv

Slots:  55
alarm_type
app_name
artist_name
audiobook_author
audiobook_name
business_name
business_type
change_amount
coffee_type
color_type
cooking_type
currency_name
date
definition_word
device_type
drink_type
email_address
email_folder
event_name
food_type
game_name
game_type
general_frequency
house_place
ingredient
joke_type
list_name
meal_type
media_type
movie_name
movie_type
music_album
music_descriptor
music_genre
news_topic
order_type
person
personal_info
place_name
player_setting
playlist_name
podcast_descriptor
podcast_name
radio_name
relation
song_name
time
time_zone
timeofday
transport_agency
transport_descriptor
transport_name
transport_type
weather_descriptor
O


In [44]:
# examples from the intent training file
! head -n 10 {DATA_DIR}/nemo_format/train.tsv

sentence	label
what alarms do i have set right now	0
checkout today alarm of meeting	0
report alarm settings	0
see see for me the alarms that you have set tomorrow morning	0
is there an alarm for ten am	0
confirm the alarm time	0
show my alarms	0
at what time have you set alarm for me	0
please list active alarms	0


In [46]:
# examples from the slot training file
! head -n 10 {DATA_DIR}/nemo_format/train_slots.tsv

54 54 54 54 54 54 54 54
54 12 54 54 54
54 54 54
54 54 54 54 54 54 54 54 54 54 12 48
54 54 54 54 54 46 46
54 54 54 54
54 54 54
54 54 54 54 54 54 54 54 54
54 54 54 54
54 54 54 54 54 54


# Training model

## Model configuration

Our Joint Intent and Slot classification model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model with an Intent and Slot Classification layer on top of it.

All model and training parameters are defined in the **intent_slot_classification_config.yaml** config file. It contains 2 main sections:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

In [5]:
# print content of the config file
MODEL_CONFIG = "intent_slot_classification_config.yaml"
config_file = f'{EXAMPLE_DIR}/conf/{MODEL_CONFIG}'
print(config_file)
config = OmegaConf.load(config_file)
print(OmegaConf.to_yaml(config))

/Users/vgetselevich/Development/NeMoOriginal/examples/nlp/intent_slot_classification/conf/intent_slot_classification_config.yaml
trainer:
  gpus: 1
  num_nodes: 1
  max_epochs: 50
  max_steps: null
  accumulate_grad_batches: 1
  amp_level: O0
  precision: 16
  distributed_backend: ddp
  row_log_interval: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
  checkpoint_callback: false
  logger: false
model:
  nemo_path: null
  data_dir: ???
  class_balancing: null
  intent_loss_weight: 0.6
  pad_label: -1
  ignore_extra_tokens: false
  ignore_start_end: true
  train_ds:
    prefix: train
    batch_size: 32
    shuffle: true
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
  validation_ds:
    prefix: test
    batch_size: 32
    shuffle: false
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
  language_model:
    pretrained_model_name: bert-base-uncased
    max_seq_length: 50
    bert_checkpoint_file: null
    bert_

## Setting up Data within the config

Among other things, the config file contains dictionaries called dataset, train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

We assume that both training and evaluation files are located in the same directory, and use the default names mentioned during the data download step. 
So, to start model training, we simply need to specify `model.data_dir`, like we are going to do below. Also notice that some config lines, including `model.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user.

`config.model.intent_loss_weight` - is a balance of training loss between Intent and Slot losses, a number between 0 to 1. It's default value is 0.6 which gives slightly higher priority to the Intent loss and it works quite well. You can experiment with this changing value if you like.

Let's now add the data directory path to the config.

In [7]:
config.model.data_dir = f'{DATA_DIR}/nemo_format'

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem. `config.trainer.max_epochs` - param defines number of training epochs. Usually 50-100 epochs or less should be enough to train on your data. Let's instantiate the Trainer object.

In [20]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

config.trainer.precision = 16 if torch.cuda.is_available() else 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = O1

# remove distributed training flags
config.trainer.distributed_backend = None

# setup a small number of epochs for demonstration purposes of this tutorial
config.trainer.max_epochs = 5

trainer = pl.Trainer(**config.trainer)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it. Model check points durng training will be saved in this directory. 

In [19]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
# the exp_dir provides a path to the current experiment for easy access
print(str(exp_dir))

[NeMo I 2020-09-11 13:48:54 exp_manager:169] Experiments will be logged at /Users/vgetselevich/Development/NeMoOriginal/tutorials/nlp/nemo_experiments/IntentSlot/2020-09-11_13-40-29
[NeMo I 2020-09-11 13:48:54 exp_manager:503] TensorboardLogger has been set up


[NeMo W 2020-09-11 13:48:54 exp_manager:537] trainer had a weights_save_path of cwd(). This was ignored.


/Users/vgetselevich/Development/NeMoOriginal/tutorials/nlp/nemo_experiments/IntentSlot/2020-09-11_13-40-29


## Initializing the model and Training

Initial statistics of the dataset will be displayed at the beginning of the training and then Intent and Slot classification report will be displayed after each training epoch.

In [13]:
# initialize the model
model = nemo_nlp.models.IntentSlotClassificationModel(config.model, trainer=trainer)

# train
trainer.fit(model)

[NeMo I 2020-09-11 13:43:31 bert_tokenizer:76] Deriving bert model type from pretrained model name.
[NeMo I 2020-09-11 13:43:31 bert_tokenizer:79] Using BertTokenizer tokenizer for bert
[NeMo I 2020-09-11 13:43:32 intent_slot_classification_descriptor:87]  Stats calculating for train mode...
[NeMo I 2020-09-11 13:43:32 intent_slot_classification_descriptor:112] Three most popular intents in train mode:
[NeMo I 2020-09-11 13:43:32 data_preprocessing:131] 0 item: 0, 175 out of 9960, 0.017570281124497992.
[NeMo I 2020-09-11 13:43:32 data_preprocessing:131] 1 item: 2, 175 out of 9960, 0.017570281124497992.
[NeMo I 2020-09-11 13:43:32 data_preprocessing:131] 2 item: 6, 175 out of 9960, 0.017570281124497992.
[NeMo I 2020-09-11 13:43:32 intent_slot_classification_descriptor:118] Three most popular slots in train mode:
[NeMo I 2020-09-11 13:43:32 data_preprocessing:131] 0 item: 54, 53689 out of 65658, 0.8177069054799111.
[NeMo I 2020-09-11 13:43:32 data_preprocessing:131] 1 item: 38, 1290 out 

[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:117] i: 0
[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:118] subtokens: [CLS] what alarms do i have set right now [SEP]
[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:119] loss_mask: 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:120] input_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:121] subtokens_mask: 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:123] slots_label: 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54 54
[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:117] i: 1
[NeMo I 2020-09-11 13:43:35 intent_slot_classification_dataset:118] subtokens: [CLS] check ##out today alarm of meeting [SEP]
[NeMo I


[NeMo I 2020-09-11 13:45:41 modelPT:559] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        eps: 1e-08
        lr: 2e-05
        weight_decay: 0.01
    )
[NeMo I 2020-09-11 13:45:41 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x148b32e90>" 
    will be used during training (effective maximum steps = 5) - 
    Parameters : 
    (last_epoch: -1
    warmup_steps: null
    warmup_ratio: 0.1
    max_steps: 5
    )
[NeMo I 2020-09-11 13:45:41 modelPT:559] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        eps: 1e-08
        lr: 2e-05
        weight_decay: 0.01
    )
[NeMo I 2020-09-11 13:45:41 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x147fa1390>" 
    will be used during training (effective maximum steps = 5) - 
    Parameters : 
    (last_epoch: -1
    warmup_steps: null
    warmup_ratio: 0.1
  

[NeMo W 2020-09-11 13:45:41 nemo_logging:349] /Users/vgetselevich/opt/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
      warnings.warn(*args, **kwargs)
    

  | Name                         | Type                    | Params
-------------------------------------------------------------------------
0 | bert_model                   | BertEncoder             | 109 M 
1 | classifier                   | SequenceTokenClassifier | 1 M   
2 | intent_loss                  | CrossEntropyLoss        | 0     
3 | slot_loss                    | CrossEntropyLoss        | 0     
4 | total_loss                   | AggregatorLoss          | 0     
5 | intent_classification_report | ClassificationReport    | 0     
6 | slot_classification_report   | ClassificationReport    | 0     
[NeMo W 2020-09-11 13:45:41 nemo_logging:349] /

[NeMo I 2020-09-11 13:45:42 classification_report:142] 
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                                0.00       0.00       0.00         19
    alarm_remove (label_id: 1)                               0.00       0.00       0.00         11
    alarm_set (label_id: 2)                                  0.00       0.00       0.00          2
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          0
    audio_volume_mute (label_id: 4)                          0.00       0.00       0.00          0
    audio_volume_up (label_id: 5)                            0.00       0.00       0.00          0
    calendar_query (label_id: 6)                             0.00       0.00       0.00          0
    calendar_remove (label_id: 7)                            0.00       0.00       0.00          0
    calendar_set (label_id: 8)               

[NeMo W 2020-09-11 13:45:42 nemo_logging:349] /Users/vgetselevich/opt/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      warnings.warn(*args, **kwargs)
    


Saving latest checkpoint..


1

After training for 5 epochs, which should take not more than few minutes, you can expect training precision for this data set to be around (the accuracy will gradually continue to improve for this data set up to about 50 epochs of training): 
```
Intents:
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                               94.74      94.74      94.74         19
    alarm_remove (label_id: 1)                             100.00     100.00     100.00         11
    alarm_set (label_id: 2)                                 85.71      94.74      90.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                        100.00      86.67      92.86         15
    audio_volume_up (label_id: 5)                           56.52     100.00      72.22         13
    calendar_query (label_id: 6)                            55.00      57.89      56.41         19
    calendar_remove (label_id: 7)                           88.89      84.21      86.49         19
    calendar_set (label_id: 8)                              81.25      68.42      74.29         19
    cooking_recipe (label_id: 9)                            86.36     100.00      92.68         19
    datetime_convert (label_id: 10)                          0.00       0.00       0.00          8
    datetime_query (label_id: 11)                           65.52     100.00      79.17         19
    email_addcontact (label_id: 12)                        100.00      12.50      22.22          8
    email_query (label_id: 13)                              83.33      78.95      81.08         19
    email_querycontact (label_id: 14)                       62.50      78.95      69.77         19
    email_sendemail (label_id: 15)                          70.83      89.47      79.07         19
    general_affirm (label_id: 16)                           95.00     100.00      97.44         19
    general_commandstop (label_id: 17)                     100.00     100.00     100.00         19
    general_confirm (label_id: 18)                         100.00     100.00     100.00         19
    general_dontcare (label_id: 19)                        100.00     100.00     100.00         19
    general_explain (label_id: 20)                         100.00      94.74      97.30         19
    general_joke (label_id: 21)                            100.00     100.00     100.00         12
    general_negate (label_id: 22)                           95.00     100.00      97.44         19
    general_praise (label_id: 23)                          100.00      94.74      97.30         19
    general_quirky (label_id: 24)                           40.00      10.53      16.67         19
    general_repeat (label_id: 25)                          100.00     100.00     100.00         19
    iot_cleaning (label_id: 26)                             84.21     100.00      91.43         16
    iot_coffee (label_id: 27)                               94.74      94.74      94.74         19
    iot_hue_lightchange (label_id: 28)                      94.44      89.47      91.89         19
    iot_hue_lightdim (label_id: 29)                        100.00      83.33      90.91         12
    iot_hue_lightoff (label_id: 30)                         89.47      89.47      89.47         19
    iot_hue_lighton (label_id: 31)                           0.00       0.00       0.00          3
    iot_hue_lightup (label_id: 32)                          81.25      92.86      86.67         14
    iot_wemo_off (label_id: 33)                             60.00     100.00      75.00          9
    iot_wemo_on (label_id: 34)                             100.00      14.29      25.00          7
    lists_createoradd (label_id: 35)                        78.95      78.95      78.95         19
    lists_query (label_id: 36)                              78.95      78.95      78.95         19
    lists_remove (label_id: 37)                             90.00      94.74      92.31         19
    music_likeness (label_id: 38)                           70.59      66.67      68.57         18
    music_query (label_id: 39)                              77.78      73.68      75.68         19
    music_settings (label_id: 40)                            0.00       0.00       0.00          7
    news_query (label_id: 41)                               77.78      73.68      75.68         19
    play_audiobook (label_id: 42)                           90.00      94.74      92.31         19
    play_game (label_id: 43)                                80.00      84.21      82.05         19
    play_music (label_id: 44)                               53.85      73.68      62.22         19
    play_podcasts (label_id: 45)                            89.47      89.47      89.47         19
    play_radio (label_id: 46)                               93.75      78.95      85.71         19
    qa_currency (label_id: 47)                              95.00     100.00      97.44         19
    qa_definition (label_id: 48)                            85.00      89.47      87.18         19
    qa_factoid (label_id: 49)                               45.16      73.68      56.00         19
    qa_maths (label_id: 50)                                100.00     100.00     100.00         14
    qa_stock (label_id: 51)                                 95.00     100.00      97.44         19
    recommendation_events (label_id: 52)                    94.44      89.47      91.89         19
    recommendation_locations (label_id: 53)                 94.74      94.74      94.74         19
    recommendation_movies (label_id: 54)                   100.00     100.00     100.00         10
    social_post (label_id: 55)                              90.00      94.74      92.31         19
    social_query (label_id: 56)                             94.74     100.00      97.30         18
    takeaway_order (label_id: 57)                           93.75      78.95      85.71         19
    takeaway_query (label_id: 58)                           85.71      94.74      90.00         19
    transport_query (label_id: 59)                          83.33      78.95      81.08         19
    transport_taxi (label_id: 60)                          100.00     100.00     100.00         18
    transport_ticket (label_id: 61)                         89.47      89.47      89.47         19
    transport_traffic (label_id: 62)                       100.00     100.00     100.00         19
    weather_query (label_id: 63)                           100.00      89.47      94.44         19
    -------------------
    micro avg                                               85.04      85.04      85.04       1076
    macro avg                                               81.13      80.81      79.36       1076
    weighted avg                                            84.10      85.04      83.54       1076
    
Slots:
    label                                                precision    recall       f1           support   
    alarm_type (label_id: 0)                                 0.00       0.00       0.00          0
    app_name (label_id: 1)                                   0.00       0.00       0.00          6
    artist_name (label_id: 2)                                0.00       0.00       0.00         21
    audiobook_author (label_id: 3)                           0.00       0.00       0.00          1
    audiobook_name (label_id: 4)                             0.00       0.00       0.00         18
    business_name (label_id: 5)                             60.00      56.60      58.25         53
    business_type (label_id: 6)                              0.00       0.00       0.00         24
    change_amount (label_id: 7)                              0.00       0.00       0.00         25
    coffee_type (label_id: 8)                                0.00       0.00       0.00          4
    color_type (label_id: 9)                                 0.00       0.00       0.00         12
    cooking_type (label_id: 10)                              0.00       0.00       0.00          0
    currency_name (label_id: 11)                            84.09      75.51      79.57         49
    date (label_id: 12)                                     57.95      91.07      70.83        112
    definition_word (label_id: 13)                           0.00       0.00       0.00         20
    device_type (label_id: 14)                              74.55      51.25      60.74         80
    drink_type (label_id: 15)                                0.00       0.00       0.00          0
    email_address (label_id: 16)                             0.00       0.00       0.00         14
    email_folder (label_id: 17)                              0.00       0.00       0.00          1
    event_name (label_id: 18)                              100.00      13.24      23.38         68
    food_type (label_id: 19)                                51.72      69.77      59.41         43
    game_name (label_id: 20)                                60.00      14.29      23.08         21
    game_type (label_id: 21)                                 0.00       0.00       0.00          0
    general_frequency (label_id: 22)                         0.00       0.00       0.00          9
    house_place (label_id: 23)                              93.33      42.42      58.33         33
    ingredient (label_id: 24)                                0.00       0.00       0.00          6
    joke_type (label_id: 25)                                 0.00       0.00       0.00          4
    list_name (label_id: 26)                                 0.00       0.00       0.00         21
    meal_type (label_id: 27)                                 0.00       0.00       0.00          0
    media_type (label_id: 28)                                0.00       0.00       0.00         37
    movie_name (label_id: 29)                                0.00       0.00       0.00          0
    movie_type (label_id: 30)                                0.00       0.00       0.00          0
    music_album (label_id: 31)                               0.00       0.00       0.00          0
    music_descriptor (label_id: 32)                          0.00       0.00       0.00          3
    music_genre (label_id: 33)                               0.00       0.00       0.00          9
    news_topic (label_id: 34)                                0.00       0.00       0.00         17
    order_type (label_id: 35)                                0.00       0.00       0.00         17
    person (label_id: 36)                                   44.86      92.31      60.38         52
    personal_info (label_id: 37)                             0.00       0.00       0.00         20
    place_name (label_id: 38)                               71.25      77.03      74.03        148
    player_setting (label_id: 39)                            0.00       0.00       0.00          1
    playlist_name (label_id: 40)                             0.00       0.00       0.00          1
    podcast_descriptor (label_id: 41)                        0.00       0.00       0.00         13
    podcast_name (label_id: 42)                              0.00       0.00       0.00          4
    radio_name (label_id: 43)                               66.67      10.53      18.18         38
    relation (label_id: 44)                                  0.00       0.00       0.00         17
    song_name (label_id: 45)                                 0.00       0.00       0.00         22
    time (label_id: 46)                                     70.27      78.20      74.02        133
    time_zone (label_id: 47)                                 0.00       0.00       0.00          9
    timeofday (label_id: 48)                                 0.00       0.00       0.00         28
    transport_agency (label_id: 49)                          0.00       0.00       0.00          9
    transport_descriptor (label_id: 50)                      0.00       0.00       0.00          0
    transport_name (label_id: 51)                            0.00       0.00       0.00          4
    transport_type (label_id: 52)                           78.38      82.86      80.56         35
    weather_descriptor (label_id: 53)                        0.00       0.00       0.00         17
    O (label_id: 54)                                        92.42      98.80      95.50       5920
    -------------------
    micro avg                                               89.10      89.10      89.10       7199
    macro avg                                               21.86      18.56      18.18       7199
    weighted avg                                            84.42      89.10      86.01       7199
```

## Training Script

If you have NeMo installed locally, you can also train the model with `examples/nlp/intent_slot_classification/intent_slot_classification.py.`

To run a training script, use:

`python intent_slot_classification.py model.data_dir=PATH_TO_DATA_DIR`